# Compiling Senator and House of Representatives Stock Trading Data


In [ ]:
import pandas as pd
import yfinance as yf

## Senate Stocks

In [ ]:
senator_trades = pd.read_csv("data/cleaned/all_transactions_senate.csv")
senator_trades = senator_trades.drop("Unnamed: 0", axis=1)
senator_trades.head()

In [ ]:
senate_stocks = yf.download(" ".join(senator_trades["ticker"].unique()),
                            start=senator_trades["transaction_date"].min(),
                            end=senator_trades["transaction_date"].max())
senate_stocks = senate_stocks.drop(["Volume", "Open", "Adj Close", "Low", "High"], axis=1)
senate_stocks

In [ ]:
trades = []
for senator in senator_trades["senator"].unique():
    data = senator_trades[senator_trades["senator"] == senator]
    data = data.sort_values("transaction_date", ascending=True)

    temp = {}

    for row in data.itertuples():
        weight = (row.amount_lower + row.amount_upper) / 2
        if row.type == "Sale (Full)":
            if row.ticker in temp and row.ticker in senate_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = senate_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = senate_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                    temp.pop(row.ticker)
                except KeyError as e:
                    print(e)
        elif row.type == "Sale (Partial)":
            if row.ticker in temp:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = senate_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = senate_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                except KeyError as e:
                    print(e)
        elif row.type == "Purchase":
            row_dict = {}
            for field in senator_trades.columns:
                row_dict[field] = getattr(row, field)
            temp[row.ticker] = row_dict

senator_trades = pd.DataFrame(trades)

In [ ]:
senator_trades.to_csv("data/cleaned/senator_trades.csv")

## House of Representatives Stocks

In [ ]:
house_trades = pd.read_csv("data/cleaned/all_transactions_house.csv")
house_trades = house_trades.drop("Unnamed: 0", axis=1)
house_trades

In [ ]:
house_stocks = yf.download(" ".join(house_trades["ticker"].unique()),
                            start=house_trades["transaction_date"].min(),
                            end=house_trades["transaction_date"].max())
house_stocks = house_stocks.drop(["Volume", "Open", "Adj Close", "Low", "High"], axis=1)
house_stocks

In [ ]:
trades = []
for representative in house_trades["representative"].unique():
    data = house_trades[house_trades["representative"] == representative]
    data = data.sort_values("transaction_date", ascending=True)
    temp = {}
    for row in data.itertuples():
        weight = (row.amount_lower + row.amount_upper) / 2
        if row.type == "sale_full":
            if row.ticker in temp and row.ticker in house_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = house_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = house_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                    temp.pop(row.ticker)
                except KeyError as e:
                    print(e)
        elif row.type == "sale_partial":
            if row.ticker in temp and row.ticker in house_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = house_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = house_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                except KeyError as e:
                    print(e)
        elif row.type == "purchase":
            row_dict = {}
            for field in house_trades.columns:
                row_dict[field] = getattr(row, field)
            temp[row.ticker] = row_dict

house_trades = pd.DataFrame(trades)

In [ ]:
house_trades.to_csv("data/cleaned/house_trades.csv")